In [1]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import time
import os
from dataclasses import dataclass
from collections import namedtuple
import pyro
import optax
from pyro.infer import SVI, TraceGraph_ELBO
import pyro.distributions as dist
import pyro.poutine as poutine
import pyro.contrib.examples.multi_mnist as multi_mnist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.font_manager as font_manager
from matplotlib.patches import Rectangle
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import seaborn as sns
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib import rcParams
from scipy.interpolate import griddata
import genjax
from genjax import grasp
from genjax import Pytree
import equinox as eqx
from genjax.typing import Any
from genjax.typing import Tuple
from genjax.typing import FloatArray
from genjax.typing import Int
from genjax.typing import IntArray
from genjax.typing import PRNGKey
from genjax.typing import typecheck
from genjax import choice_map

from numpyro.examples.datasets import MNIST
from numpyro.examples.datasets import load_dataset

import genjax
from genjax import grasp
from genjax import gensp
from genjax import select

import adevjax

console = genjax.pretty()
key = jax.random.PRNGKey(314159)
sns.set_theme(style="white")
font_path = (
    "/home/femtomc/.local/share/fonts/Unknown Vendor/TrueType/Lato/Lato_Bold.ttf"
)
font_manager.fontManager.addfont(font_path)
custom_font_name = font_manager.FontProperties(fname=font_path).get_name()
rcParams["font.family"] = custom_font_name
rcParams["figure.autolayout"] = True
label_fontsize = 70  # Set the desired font size here

train_init, train_fetch = load_dataset(MNIST, batch_size=4096, split="train")
num_train, train_idx = train_init()
data_batch = train_fetch(0)[0]
batch_sizes = [64, 128, 256, 512, 1024]

In [2]:
# Utilities for defining the model and the guide.
@dataclass
class Decoder(Pytree):
    dense_1: Any
    dense_2: Any

    def flatten(self):
        return (self.dense_1, self.dense_2), ()

    @classmethod
    def new(cls, key1, key2):
        dense_1 = eqx.nn.Linear(10, 200, key=key1)
        dense_2 = eqx.nn.Linear(200, 28 * 28, key=key2)
        return Decoder(dense_1, dense_2)

    def __call__(self, z_what):
        v = self.dense_1(z_what)
        v = jax.nn.leaky_relu(v)
        v = self.dense_2(v)
        return jax.nn.sigmoid(v)


# Create our decoder.
key, sub_key1, sub_key2 = jax.random.split(key, 3)
decoder = Decoder.new(sub_key1, sub_key2)


@dataclass
class Encoder(Pytree):
    dense_1: Any
    dense_2: Any

    def flatten(self):
        return (self.dense_1, self.dense_2), ()

    @classmethod
    def new(cls, key1, key2):
        dense_1 = eqx.nn.Linear(28 * 28, 200, key=key1)
        dense_2 = eqx.nn.Linear(200, 20, key=key2)
        return Encoder(dense_1, dense_2)

    def __call__(self, data):
        v = self.dense_1(data)
        v = jax.nn.leaky_relu(v)
        v = self.dense_2(v)
        return v[0:10], jax.nn.softplus(v[10:])


key, sub_key1, sub_key2 = jax.random.split(key, 3)
encoder = Encoder.new(sub_key1, sub_key2)

In [25]:
@genjax.gen
def model(decoder):
    latent = genjax.tfp_mv_normal_diag(jnp.zeros(10), jnp.ones(10)) @ "latent"
    logits = decoder(latent)
    _ = genjax.tfp_bernoulli(logits) @ "image"


@genjax.gen
def guide(encoder, chm):
    image = chm["image"]
    μ, Σ_scale = encoder(image)
    _ = grasp.mv_normal_diag_reparam(μ, Σ_scale) @ "latent"


def batch_elbo_grad_estimate(key, encoder, decoder, data_batch):
    def _inner(key, encoder, decoder, data):
        chm = choice_map({"image": data.flatten()})
        objective = grasp.elbo(model, guide, chm)
        return objective.grad_estimate(key, ((decoder,), (encoder, chm)))

    sub_keys = jax.random.split(key, len(data_batch))
    return jax.vmap(_inner, in_axes=(0, None, None, 0))(
        sub_keys, encoder, decoder, data_batch
    )


jit1 = jax.jit(batch_elbo_grad_estimate)

In [26]:
grasp_runtime_over_batches = []
std_ds = []
for batch_size in batch_sizes:
    print(batch_size)
    train_init, train_fetch = load_dataset(MNIST, batch_size=batch_size, split="train")
    num_train, train_idx = train_init()
    data_batch = train_fetch(0)[0]
    durations = []
    jit1(key, encoder, decoder, data_batch)
    for i in range(0, 100):
        t0 = time.perf_counter()
        jit1(key, encoder, decoder, data_batch)
        duration = time.perf_counter() - t0
        durations.append(duration)
    grasp_runtime_over_batches.append(jnp.mean(jnp.array(durations)))
    std_ds.append(jnp.std(jnp.array(durations)))

64
128
256
512
1024


In [27]:
(1000 * np.array(grasp_runtime_over_batches), 1000 * np.array(std_ds))


(
    array([0.11952992, 0.22033532, 0.31144744, 0.56776536, 1.5838046 ],
      dtype=float32),
    array([0.0298365 , 0.22446465, 0.18173508, 0.35711813, 1.1378889 ],
      dtype=float32)
)

@jax.jit
def epoch_train(params, opt_state, key, train_idx):
    def body_fn(carry, xs):
        idx, opt_state, params = carry
        updater_key = jax.random.fold_in(key, idx)
        batch = train_fetch(idx, train_idx)[0]
        ((decoder_grads,), (encoder_grads, _)) = batch_elbo_grad_estimate(
            updater_key, encoder, decoder, batch
        )
        grads = (encoder_grads, decoder_grads)
        grads = jtu.tree_map(lambda v: jnp.mean(v, axis=0), grads)
        updates, opt_state = optax.adam(1e-3).update(grads, opt_state, params)
        params = optax.apply_updates(params, updates)
        idx += 1
        return (idx, opt_state, params), ()

    idx = 0
    (_, _, params), () = jax.lax.scan(
        body_fn, (idx, opt_state, params), None, length=64
    )
    return params, 0.0


params = (encoder, decoder)
grasp_epoch_times = []
for i in range(0, 500):
    if i % 50 == 0:
        print(i)
    train_init, train_fetch = load_dataset(MNIST, batch_size=64, split="train")
    adam = optax.adam(1e-3)
    opt_state = adam.init(params)
    num_batch, train_idx = train_init()
    key, sub_key = jax.random.split(key)
    t0 = time.perf_counter()
    params, loss = epoch_train(params, opt_state, sub_key, train_idx)
    duration = time.perf_counter() - t0
    if i != 0:
        grasp_epoch_times.append(duration)

grasp_epoch_times

In [35]:
from tensorflow_probability.substrates import jax as tfp

tfd = tfp.distributions

MvNormalDiag = tfd.MultivariateNormalDiag
Bernoulli = tfd.Bernoulli

# Manual.
def batch_elbo_grad_estimate(key, encoder, decoder, data_batch):
    def single_estimate(key, encoder, decoder, data):
        image = data.flatten()

        def loss_estimate(params):
            (encoder, decoder) = params
            μ, Σ_scale = encoder(image)
            v = MvNormalDiag(jnp.zeros(10), jnp.ones(10)).sample(seed=key)
            s = μ + v * Σ_scale
            guide_normal_logp = MvNormalDiag(μ, Σ_scale).log_prob(s)
            model_normal_logp = MvNormalDiag(jnp.zeros(10), jnp.ones(10)).log_prob(s)
            logits = decoder(s)
            model_bernoulli_logp = Bernoulli(logits=logits).log_prob(image).sum()
            return (model_bernoulli_logp + model_normal_logp) - guide_normal_logp

        return jax.grad(loss_estimate)((encoder, decoder))

    sub_keys = jax.random.split(key, len(data_batch))
    return jax.vmap(single_estimate, in_axes=(0, None, None, 0))(
        sub_keys, encoder, decoder, data_batch
    )


jit2 = jax.jit(batch_elbo_grad_estimate)

@jax.jit
def epoch_train(params, opt_state, key, train_idx):
    def body_fn(carry, xs):
        idx, opt_state, params = carry
        updater_key = jax.random.fold_in(key, idx)
        batch = train_fetch(idx, train_idx)[0]
        grads = batch_elbo_grad_estimate(updater_key, encoder, decoder, batch)
        grads = jtu.tree_map(lambda v: jnp.mean(v, axis=0), grads)
        updates, opt_state = optax.adam(1e-3).update(grads, opt_state, params)
        params = optax.apply_updates(params, updates)
        idx += 1
        return (idx, opt_state, params), ()

    idx = 0
    (_, _, params), () = jax.lax.scan(
        body_fn, (idx, opt_state, params), None, length=64
    )
    return params, 0.0


params = (encoder, decoder)
hand_epoch_times = []
for i in range(0, 500):
    if i % 50 == 0:
        print(i)
    train_init, train_fetch = load_dataset(MNIST, batch_size=64, split="train")
    adam = optax.adam(1e-3)
    opt_state = adam.init(params)
    num_batch, train_idx = train_init()
    key, sub_key = jax.random.split(key)
    t0 = time.perf_counter()
    params, loss = epoch_train(params, opt_state, sub_key, train_idx)
    duration = time.perf_counter() - t0
    if i != 0:
        hand_epoch_times.append(duration)

hand_epoch_times

In [36]:
hand_runtime_over_batches = []
hand_std_ds = []
for batch_size in batch_sizes:
    print(batch_size)
    train_init, train_fetch = load_dataset(MNIST, batch_size=batch_size, split="train")
    num_train, train_idx = train_init()
    data_batch = train_fetch(0)[0]
    durations = []
    jit2(key, encoder, decoder, data_batch)
    for i in range(0, 100):
        t0 = time.perf_counter()
        jit2(key, encoder, decoder, data_batch)
        duration = time.perf_counter() - t0
        durations.append(duration)
    hand_runtime_over_batches.append(jnp.mean(jnp.array(durations)))
    hand_std_ds.append(jnp.std(jnp.array(durations)))

64
128
256
512
1024


In [37]:
(1000.0 * np.array(hand_runtime_over_batches), 1000.0 * np.array(hand_std_ds))


(
    array([0.0958897 , 0.16077422, 0.29415104, 0.54994816, 1.0759283 ],
      dtype=float32),
    array([0.03943987, 0.08780592, 0.17903097, 0.34972316, 0.7014764 ],
      dtype=float32)
)

plt.rcParams["text.usetex"] = False
fig = plt.figure(figsize=(10, 4), dpi=240)
plt.plot(
    jnp.log2(jnp.array(batch_sizes)),
    1e3 * jnp.array(grasp_runtime_over_batches),
    label="Our automation",
    color="blue",
)
plt.plot(
    jnp.log2(jnp.array(batch_sizes)),
    1e3 * jnp.array(hand_runtime_over_batches),
    label="Hand coded",
    color="red",
)
plt.title(
    "Gradient computation runtime vs. minibatch size", fontsize=label_fontsize / 3
)
plt.xlabel("Batch size (log2)", fontsize=label_fontsize / 3)
plt.ylabel("Runtime (ms)", fontsize=label_fontsize / 3)
plt.legend(fontsize=label_fontsize / 4)
plt.xticks(fontsize=label_fontsize / 5)
plt.yticks(fontsize=label_fontsize / 5)
plt.legend(fontsize=label_fontsize / 4)
fig.savefig("img/overhead_batch_size.pdf", format="pdf")
plt.show()

fig = plt.figure(figsize=(10, 4))
plt.hist(
    1e3 * jnp.array(grasp_epoch_times),
    bins=10,
    alpha=0.5,
    label="Our automation",
    color="blue",
)
plt.hist(
    1e3 * jnp.array(hand_epoch_times),
    bins=10,
    alpha=0.5,
    label="Hand coded",
    color="red",
)
plt.title("Epoch time distribution", fontsize=label_fontsize / 3)
plt.xlabel("Time (ms)", fontsize=label_fontsize / 3)
plt.ylabel("Frequency", fontsize=label_fontsize / 3)
plt.legend(fontsize=label_fontsize / 4)
plt.xticks(fontsize=label_fontsize / 5)
plt.yticks(fontsize=label_fontsize / 5)
plt.tight_layout()
fig.savefig("img/overhead_epoch_histogram.pdf", format="pdf")
plt.show()